In [1]:
import mwapi
import datetime as dt
from requests_oauthlib import OAuth1
import secrets.oauth as oauth_cfg
import re
import time
import html
import IPython.display as ipyd

# Configuration

In [2]:
# A descriptive user agent for our API requests
USER_AGENT = (
    "CE Insights survey bot -- " +
    "https://github.com/wikimedia-research/Community-Engagement-Insights-sampling"
)

# A MassMessage list for basic testing. 
TEST_LIST = "User:Neil P. Quinn-WMF/MassMessage test list"

# Number of seconds to wait before sending each individual MassMessage
WAITING_PERIOD = 15

In [3]:
MESSAGE_PAGE = "Community Engagement Insights/MassMessages/First message"

# Languages other than English with a translation available
MESSAGE_LANGS = ["ar", "de", "es", "fr", "it", "ja", "nl", "pl", "pt", "ru", "uk", "zh"]

# The name of the tag used to mark the content to send on the message page
MESSAGE_MARKER = "ce-insights-content"

# Load strata

In [8]:
real_strata = pd.read_csv("data/interim/sampled-strata.tsv", sep = "\t")
real_strata.head()

,proj_group,edit_bin,sample_size,sampled_users,page_title,survey_url,preview_url
0,wikidata,"[10, 30)",0,[],Community Engagement Insights/MassMessages/Lis...,https://wikimedia.qualtrics.com/jfe/form/SV_5A...,https://wikimedia.qualtrics.com/jfe/preview/SV...
1,wikidata,"[30, 150)",48,"[user(user='Meliade', proj_domain='www.wikidat...",Community Engagement Insights/MassMessages/Lis...,https://wikimedia.qualtrics.com/jfe/form/SV_5A...,https://wikimedia.qualtrics.com/jfe/preview/SV...
2,wikidata,"[150, 600)",64,"[user(user='Joel7687', proj_domain='www.wikida...",Community Engagement Insights/MassMessages/Lis...,https://wikimedia.qualtrics.com/jfe/form/SV_5A...,https://wikimedia.qualtrics.com/jfe/preview/SV...
3,wikidata,"[600, 1200)",35,"[user(user='RemoRivelli', proj_domain='www.wik...",Community Engagement Insights/MassMessages/Lis...,https://wikimedia.qualtrics.com/jfe/form/SV_5A...,https://wikimedia.qualtrics.com/jfe/preview/SV...
4,wikidata,"[1200, 3500)",50,"[user(user='Abián', proj_domain='www.wikidata....",Community Engagement Insights/MassMessages/Lis...,https://wikimedia.qualtrics.com/jfe/form/SV_5A...,https://wikimedia.qualtrics.com/jfe/preview/SV...


In [7]:
test_strata = pd.read_csv("data/interim/test-strata.tsv", sep = "\t")
test_strata.head()

,proj_group,edit_bin,sampled_users,survey_url,preview_url,page_title
0,arwiki,"[10, 30)","[[""Neil P. Quinn-WMF"", ""ar.wikipedia.org""], [""...",https://wikimedia.qualtrics.com/jfe/form/SV_5A...,https://wikimedia.qualtrics.com/jfe/preview/SV...,User:Neil P. Quinn/2018 CE Insights/ar1
1,asia_wps,"[30, 150)","[[""Neil P. Quinn-WMF"", ""ko.wikipedia.org""], [""...",https://wikimedia.qualtrics.com/jfe/form/SV_5A...,https://wikimedia.qualtrics.com/jfe/preview/SV...,User:Neil P. Quinn/2018 CE Insights/as2
2,cee_wps,"[150, 600)","[[""Neil P. Quinn-WMF"", ""uk.wikipedia.org""], [""...",https://wikimedia.qualtrics.com/jfe/form/SV_5A...,https://wikimedia.qualtrics.com/jfe/preview/SV...,User:Neil P. Quinn/2018 CE Insights/ce3
3,commons,"[600, 1200)","[[""Neil P. Quinn-WMF"", ""commons.wikimedia.org""...",https://wikimedia.qualtrics.com/jfe/form/SV_5A...,https://wikimedia.qualtrics.com/jfe/preview/SV...,User:Neil P. Quinn/2018 CE Insights/co4
4,dewiki,"[1200, 3500)","[[""Neil P. Quinn-WMF"", ""de.wikipedia.org""], [""...",https://wikimedia.qualtrics.com/jfe/form/SV_5A...,https://wikimedia.qualtrics.com/jfe/preview/SV...,User:Neil P. Quinn/2018 CE Insights/de5


## Add languages to strata

In [9]:
proj_langs = pd.read_csv("data/raw/project-group-languages.tsv", sep = "\t")
proj_langs.head()

,proj_group,lang
0,arwiki,ar
1,asia_wps,en
2,cee_wps,en
3,commons,en
4,dewiki,de


In [10]:
test_strata = test_strata.merge(proj_langs, how = "left", on = "proj_group")
test_strata.sample(n = 10)

,proj_group,edit_bin,sampled_users,survey_url,preview_url,page_title,lang
3,commons,"[600, 1200)","[[""Neil P. Quinn-WMF"", ""commons.wikimedia.org""...",https://wikimedia.qualtrics.com/jfe/form/SV_5A...,https://wikimedia.qualtrics.com/jfe/preview/SV...,User:Neil P. Quinn/2018 CE Insights/co4,en
16,wikidata,"[1200, 3500)","[[""Neil P. Quinn-WMF"", ""www.wikidata.org""], [""...",https://wikimedia.qualtrics.com/jfe/form/SV_5A...,https://wikimedia.qualtrics.com/jfe/preview/SV...,User:Neil P. Quinn/2018 CE Insights/dt5,en
5,enwiki,"[3500, 1100000)","[[""Neil P. Quinn-WMF"", ""en.wikipedia.org""], [""...",https://wikimedia.qualtrics.com/jfe/form/SV_5A...,https://wikimedia.qualtrics.com/jfe/preview/SV...,User:Neil P. Quinn/2018 CE Insights/en6,en
10,meaf_wps,"[1200, 3500)","[[""Neil P. Quinn-WMF"", ""he.wikipedia.org""], [""...",https://wikimedia.qualtrics.com/jfe/form/SV_5A...,https://wikimedia.qualtrics.com/jfe/preview/SV...,User:Neil P. Quinn/2018 CE Insights/me5,en
11,nlwiki,"[3500, 1100000)","[[""Neil P. Quinn-WMF"", ""nl.wikipedia.org""], [""...",https://wikimedia.qualtrics.com/jfe/form/SV_5A...,https://wikimedia.qualtrics.com/jfe/preview/SV...,User:Neil P. Quinn/2018 CE Insights/nl6,nl
0,arwiki,"[10, 30)","[[""Neil P. Quinn-WMF"", ""ar.wikipedia.org""], [""...",https://wikimedia.qualtrics.com/jfe/form/SV_5A...,https://wikimedia.qualtrics.com/jfe/preview/SV...,User:Neil P. Quinn/2018 CE Insights/ar1,ar
2,cee_wps,"[150, 600)","[[""Neil P. Quinn-WMF"", ""uk.wikipedia.org""], [""...",https://wikimedia.qualtrics.com/jfe/form/SV_5A...,https://wikimedia.qualtrics.com/jfe/preview/SV...,User:Neil P. Quinn/2018 CE Insights/ce3,en
14,ruwiki,"[150, 600)","[[""Neil P. Quinn-WMF"", ""ru.wikipedia.org""], [""...",https://wikimedia.qualtrics.com/jfe/form/SV_5A...,https://wikimedia.qualtrics.com/jfe/preview/SV...,User:Neil P. Quinn/2018 CE Insights/ru3,ru
17,zhwiki,"[3500, 1100000)","[[""Neil P. Quinn-WMF"", ""zh.wikipedia.org""], [""...",https://wikimedia.qualtrics.com/jfe/form/SV_5A...,https://wikimedia.qualtrics.com/jfe/preview/SV...,User:Neil P. Quinn/2018 CE Insights/zh6,zh
1,asia_wps,"[30, 150)","[[""Neil P. Quinn-WMF"", ""ko.wikipedia.org""], [""...",https://wikimedia.qualtrics.com/jfe/form/SV_5A...,https://wikimedia.qualtrics.com/jfe/preview/SV...,User:Neil P. Quinn/2018 CE Insights/as2,en


In [11]:
real_strata = real_strata.merge(proj_langs, how = "left", on = "proj_group")
real_strata.sample(n = 10)

,proj_group,edit_bin,sample_size,sampled_users,page_title,survey_url,preview_url,lang
2,wikidata,"[150, 600)",64,"[user(user='Joel7687', proj_domain='www.wikida...",Community Engagement Insights/MassMessages/Lis...,https://wikimedia.qualtrics.com/jfe/form/SV_5A...,https://wikimedia.qualtrics.com/jfe/preview/SV...,en
8,zhwiki,"[150, 600)",596,"[user(user='HFWMan', proj_domain='zh.wikipedia...",Community Engagement Insights/MassMessages/Lis...,https://wikimedia.qualtrics.com/jfe/form/SV_5A...,https://wikimedia.qualtrics.com/jfe/preview/SV...,zh
4,wikidata,"[1200, 3500)",50,"[user(user='Abián', proj_domain='www.wikidata....",Community Engagement Insights/MassMessages/Lis...,https://wikimedia.qualtrics.com/jfe/form/SV_5A...,https://wikimedia.qualtrics.com/jfe/preview/SV...,en
9,zhwiki,"[600, 1200)",230,"[user(user='創造未來，迎接康莊', proj_domain='zh.wikipe...",Community Engagement Insights/MassMessages/Lis...,https://wikimedia.qualtrics.com/jfe/form/SV_5A...,https://wikimedia.qualtrics.com/jfe/preview/SV...,zh
3,wikidata,"[600, 1200)",35,"[user(user='RemoRivelli', proj_domain='www.wik...",Community Engagement Insights/MassMessages/Lis...,https://wikimedia.qualtrics.com/jfe/form/SV_5A...,https://wikimedia.qualtrics.com/jfe/preview/SV...,en
6,zhwiki,"[10, 30)",106,"[user(user='Qead', proj_domain='zh.wikipedia.o...",Community Engagement Insights/MassMessages/Lis...,https://wikimedia.qualtrics.com/jfe/form/SV_5A...,https://wikimedia.qualtrics.com/jfe/preview/SV...,zh
11,zhwiki,"[3500, 1100000)",206,"[user(user='Mico121', proj_domain='zh.wikipedi...",Community Engagement Insights/MassMessages/Lis...,https://wikimedia.qualtrics.com/jfe/form/SV_5A...,https://wikimedia.qualtrics.com/jfe/preview/SV...,zh
0,wikidata,"[10, 30)",0,[],Community Engagement Insights/MassMessages/Lis...,https://wikimedia.qualtrics.com/jfe/form/SV_5A...,https://wikimedia.qualtrics.com/jfe/preview/SV...,en
10,zhwiki,"[1200, 3500)",215,"[user(user='Wanchens', proj_domain='zh.wikiped...",Community Engagement Insights/MassMessages/Lis...,https://wikimedia.qualtrics.com/jfe/form/SV_5A...,https://wikimedia.qualtrics.com/jfe/preview/SV...,zh
7,zhwiki,"[30, 150)",523,"[user(user='Hoàng Tích Tùng', proj_domain='zh....",Community Engagement Insights/MassMessages/Lis...,https://wikimedia.qualtrics.com/jfe/form/SV_5A...,https://wikimedia.qualtrics.com/jfe/preview/SV...,zh


# Set up connection

Use OAuth to authenticate as an [owner-only consumer](https://www.mediawiki.org/wiki/OAuth/Owner-only_consumers).

In [12]:
sess = mwapi.Session("https://meta.wikimedia.org", user_agent = USER_AGENT)

auth = OAuth1(
    oauth_cfg.consumer_token,
    oauth_cfg.consumer_secret,
    oauth_cfg.access_token,
    oauth_cfg.access_secret
)

def get_token():
    resp = sess.get(
        action="query", 
        meta="tokens", 
        type="csrf", 
        auth = auth
    )
    
    return resp["query"]["tokens"]["csrftoken"]

def api_get(*args, **kwargs):
    return sess.get(
        *args,
        format = "json",
        formatversion = 2,
        auth = auth,
        **kwargs
    )
    
def api_post(*args, **kwargs):
    return sess.post(
        *args,
        format = "json",
        formatversion = 2,
        auth = auth,
        token = get_token(),
        **kwargs
    )

# Check API authentication 

In [13]:
api_get(
    action = "query",
    meta = "userinfo"
)["query"]["userinfo"]["name"]

'WMF Surveys'

# Assemble message

This uses the ["Assemble multilingual message" Lua module](https://meta.wikimedia.org/wiki/Module:Assemble_multilingual_message), 
as described at [meta:Newsletters/Translation#Regular process](https://meta.wikimedia.org/wiki/Newsletters/Translation#Regular_process).

In [14]:
# Be careful with the arrangement of linebreaks and pipes between the languages.
# In some cases, this can cause the leading or trailing language not to be included in the output.
message_assembly = """
{{{{#invoke:
Assemble multilingual message|
assembleMessage|
marker={marker}|
page={page}|{langs}
}}}}
""".format(
    page = MESSAGE_PAGE,
    marker = MESSAGE_MARKER,
    langs = "|".join(MESSAGE_LANGS) + "|"
)

print(message_assembly)


{{#invoke:
Assemble multilingual message|
assembleMessage|
marker=ce-insights-content|
page=Community Engagement Insights/MassMessages/First message|ar|de|es|fr|it|ja|nl|pl|pt|ru|uk|zh|
}}



In [15]:
raw_msg = api_get(
    action = "parse",
    text = message_assembly,
    prop = "text",
    contentmodel = "wikitext",
    disablelimitreport = True
)["parse"]["text"]

In [16]:
# Strip ALL instances of `<pre>` and `</pre>` from the message
# Replace HTML entities (mainly angle brackets) with raw characters
def strip_pre(msg):
    return re.sub(
        r"<[/]?pre>",
        "",
        msg
    )

# Convert the date-only signature to a date-and-name signature
def convert_sig(msg):
    return re.sub(
        r"~~~~~",
        """<span class="mw-content-ltr" dir="ltr">[[m:User:WMF Surveys|WMF Surveys]]</span>, ~~~~~""",
        msg
    )

unlinked_msg = html.unescape(strip_pre(convert_sig(raw_msg)))

In [17]:
print(unlinked_msg)

<div class="mw-parser-output">
{{subst:#switch:{{subst:CONTENTLANG}}|it=<div class="plainlinks mw-content-ltr" lang="it" dir="ltr">
Ciao! La Fondazione Wikimedia chiede il tuo parere con un sondaggio. Vorremmo sapere se stiamo sostenendo efficacemente il tuo lavoro all'interno e all'esterno del mondo wiki, e come possiamo cambiare o migliorare le cose in futuro. Le opinioni che condividi influiranno direttamente sull'attuale e futuro lavoro della Fondazione Wikimedia. Sei stato selezionato in modo casuale per partecipare a questo sondaggio poiché vorremmo ascoltare l'opinione della tua comunità Wikimedia. Il sondaggio è disponibile in diverse lingue e necessita tra i venti e i quaranta minuti per essere completato.
 
<big>'''[https://www.example.com Inizia il sondaggio adesso!]'''</big>

È possibile trovare maggiori informazioni sul progetto in [[m:Special:MyLanguage/Community_Engagement_Insights/About_CE_Insights|questa pagina]]. Il sondaggio è ospitato da un servizio terzo ed è regol

In [18]:
# Replace links
def add_link(msg, link):
    return re.sub(
        r"https://www.example.com",
        link,
        msg
    )

## Grab headings

In [19]:
def get_first_heading(lang):
    # This assumes the section heading we want is the first one on the page
    return api_get(
        action = "parse",
        page = MESSAGE_PAGE + "/" + lang,
        prop = "sections"
    )["parse"]["sections"][0]["line"]

In [20]:
titles = {}

titles["en"] = get_first_heading("en")

for lang in MESSAGE_LANGS:
    titles[lang] = get_first_heading(lang)
    
titles

{'ar': 'شارك خبراتك وآرائك وملاحظاتك معنا بصفتك ويكيميدي في هذا الاستبيان العالمي',
 'de': 'Teile deine Erfahrungen und gib uns Feedback als Wikimedianer in dieser globalen Umfrage',
 'en': 'Share your experience and feedback as a Wikimedian in this global survey',
 'es': 'Comparta su experiencia y retroalimente como wikimedista en esta encuesta global',
 'fr': 'Partagez votre expérience de wikimédien·ne dans cette enquête générale',
 'it': 'Condividi la tua esperienza e le tue valutazioni come Wikimediano tramite questo sondaggio globale',
 'ja': 'このアンケートを通してウィキメディアンとしてのご意見、ご感想をお寄せください。',
 'nl': 'Deel je feedback en je ervaringen als Wikimediabewerker in een wereldwijde enquête',
 'pl': 'Podziel się swoim doświadczeniem i przekaż opinię jako Wikimedianin w globalnej ankiecie',
 'pt': 'Compartilhe suas experiências e comentários como wikimedista na nossa nova pesquisa',
 'ru': 'Всемирный опрос для участников проектов Викимедиа — поделитесь своим мнением и опытом',
 'uk': 'Поділіться св

# Set up messaging

In [21]:
def send_message(target_list, subj, msg):
    api_post(
        action = "massmessage",
        spamlist = target_list,
        subject = subj,
        message = msg
    )


def message_frame(frame):
    for row in frame.itertuples():
        if row.sample_size > 0:
            message_list = row.page_title
            survey_url = row.survey_url
            preview_url = row.preview_url
            title_lang = row.lang

            linked_msg = add_link(unlinked_msg, survey_url)

            print_err(
                "Sending to {message_list} with url {survey_url} in {wait} seconds".format(
                    message_list = message_list,
                    survey_url = survey_url,
                    wait = WAITING_PERIOD
                )
            )

            time.sleep(WAITING_PERIOD)

            send_message(message_list, titles[title_lang], linked_msg)

            print_err("Sent.")

# Test

## Basic test

In [ ]:
print(unlinked_msg)

In [ ]:
send_message(
    TEST_LIST,
    "Testing MassMessage",
    unlinked_msg
)

## Test with full message

In [ ]:
message_frame(test_strata)

# ACTUAL SENDING

In [22]:
# Run this cell and follow the prompt to unlock sending code for 30 seconds
ipyd.HTML("""
<script type="text/Javascript">
var kernel = IPython.notebook.kernel;

if (prompt("The code below will send talk pages messages to thousands of editors. Type 'run' to unlock it")) {
    kernel.execute("allow_run = dt.datetime.now()")
};
</script
""")

In [23]:
# If the cell above has been run within the past 30 seconds, send the messages
if (dt.datetime.now() - allow_run) < dt.timedelta(seconds = 30):
    message_frame(real_strata)
else:
    print_err(
        "This code will send talk page messages to thousands of editors.", 
        "If you're certain you want to do this, run the cell above to unlock it.",
        sep = "\n"
    )

Sending to Community Engagement Insights/MassMessages/Lists/2018/dt2 with url https://wikimedia.qualtrics.com/jfe/form/SV_5ABs6WwrDHzAeLr?aud=AE&prj=dt&edc=2&prjedc=dt2 in 15 seconds
Sent.
Sending to Community Engagement Insights/MassMessages/Lists/2018/dt3 with url https://wikimedia.qualtrics.com/jfe/form/SV_5ABs6WwrDHzAeLr?aud=AE&prj=dt&edc=3&prjedc=dt3 in 15 seconds
Sent.
Sending to Community Engagement Insights/MassMessages/Lists/2018/dt4 with url https://wikimedia.qualtrics.com/jfe/form/SV_5ABs6WwrDHzAeLr?aud=VAE&prj=dt&edc=4&prjedc=dt4 in 15 seconds
Sent.
Sending to Community Engagement Insights/MassMessages/Lists/2018/dt5 with url https://wikimedia.qualtrics.com/jfe/form/SV_5ABs6WwrDHzAeLr?aud=VAE&prj=dt&edc=5&prjedc=dt5 in 15 seconds
Sent.
Sending to Community Engagement Insights/MassMessages/Lists/2018/dt6 with url https://wikimedia.qualtrics.com/jfe/form/SV_5ABs6WwrDHzAeLr?aud=VAE&prj=dt&edc=6&prjedc=dt6 in 15 seconds
Sent.
Sending to Community Engagement Insights/MassMessage